In [2]:
import pandas as pd
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import geopandas as gpd # To create GeodataFrame

In [3]:
df_neighborhoods = pd.read_pickle("../pickles/utseus-cendus-shanghai-neighborhoods.pk")

In [4]:
df_neighborhoods.head()

,NAME,TYPE,geometry
0,??,2,POLYGON Z ((121.4216787977396 30.6847124112995...
1,???,2,POLYGON Z ((121.4119888989075 30.6937387337238...
2,???,2,POLYGON Z ((121.4016307330431 30.7071734919907...
3,????,2,POLYGON Z ((121.3368669670694 30.7380603821550...
4,????,2,POLYGON Z ((121.2514316037695 30.7917653095429...


In [5]:
crs = {'init': 'epsg:4479'}

In [6]:
gdf = gpd.GeoDataFrame(df_neighborhoods, crs=crs, geometry=df_neighborhoods.geometry)

In [7]:
gdf.plot()

## How many POI of each type are in every Polygon?

In [8]:
df_POI_geom = pd.read_pickle("../pickles/savedDF_POI_geom.pkl")
df_POI_geom_resto = df_POI_geom[df_POI_geom['category']=='Restaurant']

In [9]:
df_POI_geom_resto.head()

,NAME,GPSX,GPSY,GBCODE,category,subcategory,subsubcategory,geometry
42654,佳都大厦停车场,121.429483,31.216387,110100.0,Restaurant,Chinese Restaurant,Chinese Restaurant,POINT (121.429483 31.216387)
42655,爵士岛咖啡(殷高西路店)停车场,121.479120,31.322139,110503.0,Restaurant,Leisure Restaurant,Café,POINT (121.47912 31.322139)
94385,天天农家乐,121.465817,31.682416,110100.0,Restaurant,Chinese Restaurant,Chinese Restaurant,POINT (121.465817 31.682416)
94386,缘辉农家乐,121.467229,31.681350,110100.0,Restaurant,Chinese Restaurant,Chinese Restaurant,POINT (121.467229 31.68135)
94387,上海崇明岛海岛人家农家乐(美食),121.878043,31.335275,110100.0,Restaurant,Chinese Restaurant,Chinese Restaurant,POINT (121.878043 31.335275)


In [10]:
from shapely.geometry import Point, Polygon

In [11]:
nb_restaurant = {}

In [12]:
total = 0
for index, polygon in df_neighborhoods['geometry'].items():
    if index in nb_restaurant:
        continue 
    for index2, point in df_POI_geom_resto['geometry'].items():
        if point.within(polygon) == True:
            total += 1
    nb_restaurant[index] = total
    total = 0

KeyboardInterrupt: 

In [ ]:
df_neighborhoods['restaurant'] = nb_restaurant.values()

## Bicycle park

In [ ]:
df_POI_geom_transport = df_POI_geom[df_POI_geom['category']=='Transportation']
df_POI_geom_transport.head()

In [ ]:
df_bicycle_park = df_POI_geom[df_POI_geom['subsubcategory']=='Bicycle park']

In [ ]:
nb_bicyclePark = {}
total = 0
for index, polygon in df_neighborhoods['geometry'].items():
    for index2, point in df_bicycle_park['geometry'].items():
        if point.within(polygon) == True:
            total += 1
    nb_bicyclePark[index] = total
    total = 0

In [ ]:
df_neighborhoods['Bicycle_park'] = nb_bicyclePark.values()

In [ ]:
df_neighborhoods.head()

In [107]:
df_neighborhoods.to_pickle("../pickles/savedDF_neighborhoods.pickle")

In [134]:
def nb_per_district(dataframe):
    nb = {}
    total = 0
    for index, polygon in df_neighborhoods['geometry'].items():
        if index in nb:
            continue 
        for index2, point in dataframe['geometry'].items():
            if point.within(polygon) == True:
                total += 1
        nb[index] = total
        total = 0
    return nb

## Bus terminal

In [140]:
df_bus_terminal = df_POI_geom[df_POI_geom['subsubcategory']=='Bus Terminal']

In [142]:
bus = nb_per_district(df_bus_terminal)

In [144]:
df_neighborhoods['Bus'] = bus.values()

## Railway station entrance and exit

In [148]:
df_railway = df_POI_geom[df_POI_geom['subsubcategory']=='Railway station entrance and exit']

In [149]:
railway = nb_per_district(df_railway)

In [151]:
df_neighborhoods['Railway_station_entrance_exit'] = railway.values()

## Convenience Store

In [157]:
df_shop = df_POI_geom[df_POI_geom['category']=='Shopping']

In [161]:
df_convenience_store = df_shop[df_shop['subsubcategory']=="Convenience Store"]

In [163]:
len(df_convenience_store)

26533

In [164]:
cv = nb_per_district(df_convenience_store)

In [166]:
df_neighborhoods['Convenience Store'] = cv.values()

## Scenic spot

In [169]:
df_scenic_spot = df_POI_geom[df_POI_geom['category']=='Scenic spot']

In [171]:
scenic_spot = nb_per_district(df_scenic_spot)

In [173]:
df_neighborhoods['Scenic_spot'] = scenic_spot.values()

In [181]:
df_neighborhoods.to_pickle("../pickles/savedDF_POI_per_district.pickle")

## Sport and leisure

In [176]:
df_sport = df_POI_geom[df_POI_geom['category']=='Sports and leisure']

In [177]:
sport = nb_per_district(df_sport)

In [179]:
df_neighborhoods['Sport'] = sport.values()